In [ ]:
# set up and test simple commands with LLM 
# swaped OpenAI for Mistral due to expired free tier quota

In [24]:
# import mistral 
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [25]:
import os

In [26]:
# get API key stored in env variable
api_key = os.getenv("MISTRAL_KEY")

In [28]:
client = MistralClient(api_key = api_key)

In [6]:
# ask it a question
response = client.chat(
    model = 'mistral-tiny',
    messages = [ChatMessage(role = "user", content = "Is it too late to join the course?")]
)

In [7]:
print(response.choices[0].message.content)

I'm sorry for the confusion, but I'll need a bit more information to provide an accurate answer. The phrase "the course" is quite general. Are you referring to an online course, a university course, a language class, or something else entirely? Each type of course may have different enrollment policies. Some allow late enrollments, while others do not. If you provide more details, I'll be better equipped to help.


In [7]:
# set up a search using minisearch from Alexei Grigorev
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-30 19:35:37--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-30 19:35:37 (8.97 MB/s) - ‘minsearch.py.1’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
import json

In [10]:
# get the document
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-06-30 19:35:43--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json.1’

documents.json.1    100%[===================>] 642.90K  --.-KB/s    in 0.02s   

2024-06-30 19:35:43 (26.7 MB/s) - ‘documents.json.1’ saved [658332/658332]



In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [12]:
index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields = ["course"]
)

In [13]:
# index (fit the index to) the documents
index.fit(documents)

In [14]:
# some query (question) 
query = 'the course has already started, can I still enroll?'

In [8]:
# let's search but make some fields more important and some less so
boost = {'question': 3.0, 'section': 0.5}
results = index.search(
    query,
    boost_dict = boost, 
    # filter_dict = {'course': 'data-engineering-zoomcamp'}, # if we want to search only for specific course 
    num_results = 5
)

In [9]:
print(results)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}, {'text': 'Yes, we will keep all the materials after the course finishes, s

In [ ]:
# combine LLM and search

In [18]:
# create the prompt template
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT: {context}
""".strip()

In [19]:
# define the context
context = ""
for doc in results:
    context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [21]:
prompt = prompt_template.format(question = query, context = context).strip()

In [24]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: the course has already started, can I still enroll?

CONTEXT: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your fi

In [11]:
# feed it to LLM
response = client.chat(
    model = "mistral-large-latest",
    messages = [ChatMessage(role = "user", content = prompt)]
)
print(response.choices[0].message.content)

NameError: name 'prompt' is not defined

In [ ]:
# clean up the flow

In [29]:
# define a function for search
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    search_results = index.search(
        query,
        boost_dict = boost, 
        # filter_dict = {'course': 'data-engineering-zoomcamp'}, # if we want to ask only about one specific course 
        num_results = 10
    )
    return search_results
    

In [39]:
# define a function for building a prompt
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE

    QUESTION: {question}

    CONTEXT: {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question = query, context = context).strip()
    return prompt
    

In [31]:
# define a function to talk to LLM
def llm(prompt):
    response = client.chat(
    model = "mistral-large-latest",
    messages = [ChatMessage(role = "user", content = prompt)]
    )    
    return response.choices[0].message.content

In [32]:
# call these functions within in one other function
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [33]:
print(rag('the course has already started, can I enroll now?'))

NameError: name 'index' is not defined

In [ ]:
# replace Alexey's search with elasticsearch

In [6]:
from elasticsearch import Elasticsearch

In [8]:
# create elastic search client
es_client = Elasticsearch('http://localhost:9200')

In [9]:
# check that it worked and retrieve info for hw1 Q1
es_client.info()['version']['build_hash']

'42f05b9372a9a4a470db3b52817899b99a76ee73'

In [34]:
# create an index
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} # we use course names for filtering
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index = index_name, body = index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/e2B-6M_JSE6RmyHqjTJ-YQ] already exists')

In [11]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# index the docs
for doc in documents:
    es_client.index(index = index_name, document = doc)

In [15]:
query = 'I just discovered the course. Can I still join it?'

In [40]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"], # ^3 - makes this field 3x important
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    result_docs = [hit['_source'] for hit in response['hits']['hits']]
    return result_docs

In [41]:
search_results = elastic_search(query)
print(search_results)

[{'text': "Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do this by executing the provided installation command: !pip install dlt[duckdb]. If you’re doing it locally, be sure to also have duckdb pip installed (even before the duckdb package is loaded).", 'section': 'Workshop 1 - dlthub', 'question': 'How do I install the necessary dependencies to run the code?', 'course': 'data-engineering-zoomcamp'}, {'text': "Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do this by executing the provided installation command: !pip install dlt[duckdb]. If you’re doing it locally, be sure to also have duckdb pip installed (even before the duckdb package is loaded).", 'section': 'Workshop 1 - dlthub', 'question': 'How do I install the necessary dependencies to run the code?', 'course': 'data-engineering-zoomcamp'}, {'text': "Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed.

In [42]:
# adjust rag function
def rag_es(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [44]:
rag_es("The course has already started, can I still join?")

"Yes, you can still join the course after it has started. You will be eligible to submit homeworks even if you don't register immediately. However, please be aware that there will be deadlines for turning in the final projects."

In [ ]:
# TODO: HW